<a href="https://colab.research.google.com/github/Rodrigo-Perico/DataScience/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import sklearn
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

In [31]:
import kagglehub
import pandas as pd
import os

path = kagglehub.dataset_download("renatosn/sao-paulo-housing-prices")

files = os.listdir(path)
print("Arquivos no dataset:", files)

csv_file = [f for f in files if f.endswith(".csv")][0]
csv_path = os.path.join(path, csv_file)

df = pd.read_csv(csv_path)

print(df.head())


Arquivos no dataset: ['data.csv']
              address      district  area  bedrooms  garage                type  rent  total
0          Rua Herval    Belenzinho    21         1       0     Studio e kitnet  2400   2939
1  Avenida São Miguel  Vila Marieta    15         1       1     Studio e kitnet  1030   1345
2    Rua Oscar Freire     Pinheiros    18         1       0         Apartamento  4000   4661
3    Rua Júlio Sayago       Vila Ré    56         2       2  Casa em condomínio  1750   1954
4  Rua Barata Ribeiro    Bela Vista    19         1       0     Studio e kitnet  4000   4654


In [32]:
ordinal_encoder = OrdinalEncoder()
adrress_encoder = ordinal_encoder.fit_transform([df["address"]])

In [33]:
adrress_encoder

array([[0., 0., 0., ..., 0., 0., 0.]])

In [34]:
from sklearn.preprocessing import OneHotEncoder


In [35]:
df = pd.get_dummies(df, columns=["area"])


print(df)

                               address                 district  ...  area_570  area_580
0                           Rua Herval               Belenzinho  ...     False     False
1                   Avenida São Miguel             Vila Marieta  ...     False     False
2                     Rua Oscar Freire                Pinheiros  ...     False     False
3                     Rua Júlio Sayago                  Vila Ré  ...     False     False
4                   Rua Barata Ribeiro               Bela Vista  ...     False     False
...                                ...                      ...  ...       ...       ...
11652          Avenida Adolfo Pinheiro              Santo Amaro  ...     False     False
11653  Rua Professor Alexandre Correia     Jardim Vitória Régia  ...     False     False
11654                 Rua Abílio Borin         Jardim Caravelas  ...     False     False
11655                Rua Forte William  Jardim Fonte do Morumbi  ...     False     False
11656              Ru

In [36]:
df.drop(columns=['address', 'district', 'type'], inplace=True)

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [38]:
Y = df["rent"]

X = df.loc[:, df.columns != "rent"]

In [39]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [40]:
from sklearn.linear_model import LinearRegression

lin = LinearRegression()

lin.fit(x_train, y_train)

LinearRegression()

In [41]:
from sklearn.tree import DecisionTreeRegressor

In [42]:
tree = DecisionTreeRegressor()

In [43]:
tree.fit(x_train, y_train)

DecisionTreeRegressor()

In [44]:
pred = tree.predict(x_train)

In [45]:
pred

array([2999.,  990., 1960., ..., 2100., 2244., 1850.])

In [46]:
tree_mse = mean_squared_error(y_train, pred)

In [47]:
tree_mse

194.43963721044247

In [48]:
import numpy as np
tree_rmse = np.sqrt(tree_mse)


In [49]:
tree_rmse

13.944161402194199

In [50]:
from sklearn.model_selection import cross_val_score

score = cross_val_score(tree, x_train, y_train, scoring='neg_mean_squared_error', cv=10)

In [51]:
tree_rmse_score = np.sqrt(-score)

In [52]:
print("Score:", score)
print("Mean:", score.mean())
print("Standad Deviation:", score.std())

Score: [-396847.88633578 -447041.04044118 -461107.17892157 -410115.5625
 -505497.32628676 -488278.82169118 -552413.4810049  -444548.36519608
 -379769.15196078 -392137.05521472]
Mean: -447775.5869552959
Standad Deviation: 52766.007521082785


In [53]:
from sklearn.model_selection import GridSearchCV

In [59]:
param_grid = [

    {'splitter' :['best', 'random'],  'max_depth' : [1,5,10,20,50]},
    {'min_samples_split' : [1,2,3,9,15], 'min_samples_leaf' : [1,2,5,10,20]}

]

tree = DecisionTreeRegressor()

grid_search = GridSearchCV(tree, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

grid_search.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
25 fits failed out of a total of 175.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid=[{'max_depth': [1, 5, 10, 20, 50],
                          'splitter': ['best', 'random']},
                         {'min_samples_leaf': [1, 2, 5, 10, 20],
                          'min_samples_split': [1, 2, 3, 9, 15]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [63]:
final_model = grid_search.best_estimator_

final_model_predicts = final_model.predict(x_test)

final_mse = mean_squared_error(y_test, final_model_predicts)

print(np.sqrt(final_mse))

523.9508949737208
